In [1]:
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')

In [3]:
file_name = 'F01/Session1/wav_arrayMic/0001.wav'
y, sr = librosa.load(file_name, duration=5.0)
ps = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
ps.shape

(128, 216)

In [4]:
file_name = 'F01/Session1/wav_arrayMic/0001.wav'
y, sr = librosa.load(file_name, duration=5.0)
ps = librosa.feature.melspectrogram(y=y, sr=sr)
ps.shape

(128, 216)

In [5]:
import os
import numpy as np
from tqdm import tqdm

D_sptg = [] # Dataset
D_mfcc = []

directories =[  
                 ['F01','Atypical'],['F03','Atypical'],['F04','Atypical'],['FC01','Typical'],['FC02','Typical'],
                 ['FC03','Typical'],['M01','Atypical'],['M02','Atypical'],['M03','Atypical'],['M04','Atypical'],
                 ['M05','Atypical'],['MC01','Typical'],['MC02','Typical'],['MC03','Typical'],['MC04','Typical']
             ]

for pair in directories:
    directory = pair[0]
    label = pair[1]
    print(directory)
    for filename in os.listdir(directory):
        if filename != 'Notes':
            file = os.path.join(directory, filename)
            file = os.path.join(file, 'wav_arrayMic')
            if os.path.isdir(file):
                for audiofile in tqdm(os.listdir(file)):
                    f = os.path.join(file, audiofile)
                    if os.path.isfile(f):
                        if librosa.get_duration(filename=f) != 0:
                            y, sr = librosa.load(f, duration=5.0)
                            spectogram = librosa.feature.melspectrogram(y=y, sr=sr)
                            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
                            if mfcc.shape == (128, 216) and spectogram.shape == (128,216): 
                                D_sptg.append( (spectogram, label) )
                                D_mfcc.append( (mfcc, label) )

  0%|          | 0/134 [00:00<?, ?it/s]

F01


  0%|          | 0/204 [00:00<?, ?it/s]

F03


  0%|          | 0/199 [00:00<?, ?it/s]

...

  0%|          | 0/600 [00:00<?, ?it/s]

MC03


  0%|          | 0/648 [00:00<?, ?it/s]

MC04


100%|██████████| 373/373 [00:48<00:00,  7.69it/s]


In [16]:
print("Number of samples: ", len(D_sptg), len(D_mfcc))

Number of samples:  1004 1004


In [26]:
dataset = D_sptg
random.shuffle(dataset)

train = dataset[:800]
dataset = dataset[800:]
validate = dataset[:150]
test = dataset[150:]

X_train, y_train = zip(*train)
X_validate, y_validate = zip(*validate)
X_test, y_test = zip(*test)

In [27]:
X_train = np.array(X_train)
X_validate = np.array(X_validate)
X_test = np.array(X_test)

In [28]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
#y=to_categorical(labelencoder.fit_transform(y))
#y
y_train = np.array(to_categorical(labelencoder.fit_transform(y_train)))
y_validate = np.array(to_categorical(labelencoder.fit_transform(y_validate)))
y_test = np.array(to_categorical(labelencoder.fit_transform(y_test)))

In [29]:
X_train.shape, X_validate.shape, X_test.shape, y_train.shape, y_validate.shape, y_test.shape

((800, 128, 216), (150, 128, 216), (54, 128, 216), (800, 2), (150, 2), (54, 2))

In [32]:
import tensorflow.keras as keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from tensorflow.keras.layers import LSTM, Dense

input_shape = (X_train.shape[1], X_train.shape[2])
model = keras.Sequential()
model.add(LSTM(128,input_shape=input_shape))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               176640    
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 48)                3120      
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 48)                0

In [33]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train, 
	y=y_train,
    epochs=12,
    batch_size=32,
    validation_data= (X_validate, y_validate))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
25/25 [==============================] - 4s 91ms/step - loss: 0.6476 - accuracy: 0.5838 - val_loss: 0.6122 - val_accuracy: 0.6467
Epoch 2/12
25/25 [==============================] - 2s 72ms/step - loss: 0.5142 - accuracy: 0.7588 - val_loss: 0.5277 - val_accuracy: 0.7867
Epoch 3/12
25/25 [==============================] - 2s 69ms/step - loss: 0.4425 - accuracy: 0.8050 - val_loss: 0.5051 - val_accuracy: 0.8067
Epoch 4/12
25/25 [==============================] - 2s 73ms/step - loss: 0.4104 - accuracy: 0.8150 - val_loss: 0.4959 - val_accuracy: 0.8000
Epoch 5/12
25/25 [==============================] - 2s 72ms/step - loss: 0.3820 - accuracy: 0.8313 - val_loss: 0.4643 - val_accuracy: 0.8133
Epoch 6/12
25/25 [==============================] - 2s 69ms/step - loss: 0.3647 - accuracy: 0.8450 - val_loss: 0.7783 - val_accuracy: 0.7600
Epoch 7/12
25/25 [==============================] - 2s 72ms/step - loss: 0.4571 - accuracy: 0.8250 - val_loss: 0.5213 - val_accuracy: 0.7200
Epoch 8/12
25

In [34]:
predict = model.predict_classes(X_test)
prediction_class = labelencoder.inverse_transform(predict)
#real_class = labelencoder.inverse_transform(y_test)
#prediction_class

x, y = zip(*test)
y
match = 0
print('  Real       Prediction       Match')
for i in range(0,X_test.shape[0]):
    if(y[i]==prediction_class[i]):
        match+=1
    print(y[i] + '       ' + prediction_class[i] + '       ' + str(y[i]==prediction_class[i]))
print('Matched = ', match)
print('Not Matched = ',(X_test.shape[0] - match))

  Real       Prediction       Match
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Typical       Atypical       False
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Typical       False
Typical       Typical       True
Typical     